In [1]:
# Data comes from 2015 Residential Energy Consumption (RECS) Survey 
# URL: https://www.eia.gov/consumption/residential/data/2015/index.php?view=consumption#undefined
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import Spectral10
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap
import pandas as pd # the module for opening a .xlsx file
import urllib  # the module for reading a url
import csv  # the module for opening a .csv format file



# read a .xlsx format file with a given URL
onlinefile_excel = urllib.request.urlopen("https://www.eia.gov/consumption/residential/data/2015/c&e/ce5.1a.xlsx")
data= pd.ExcelFile(onlinefile_excel)
data.sheet_names
df1 = data.parse("kWh")
#df1=df1.drop([0,1,2,3,4],axis=0)
#df1['Area']=['Northeast','Northeast','Northeast','Northeast','Northeast','Northeast','Northeast','Northeast','Midwest','Midwest', 'Midwest','South','South','South','South','West','West','West','West','West']
#Eliminate First three rows othe data frame
df2 = df1.iloc[5:20]
df2

,Release date: May 2018,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
5,Northeast,20.3,4.9,17.7,0.5,N,2.3,4.7,1.8,21,0.8,7.6,20.8
6,New England,3.5,1.4,2.8,0.1,N,0.6,1.2,0.5,5.6,0.2,2.3,5.2
7,Middle Atlantic,16.8,3.5,15,0.4,N,1.7,3.5,1.3,15.4,0.6,5.3,15.6
8,Midwest,38.5,9,28.4,1.3,N,4.7,5.9,3,30,1.4,12.2,30.4
9,East North Central,24.7,6.5,18,0.8,N,2.8,4.3,2,19.9,0.9,7.4,19.3
10,West North Central,13.8,2.5,10.4,0.5,N,1.9,1.7,1.1,10.1,0.4,4.8,11
11,South,98,4.4,140.3,6.7,0.9,12.1,3.9,1.8,95,2.3,26.5,54.2
12,South Atlantic,47.2,2.4,69.9,3.4,0.6,5.9,3.1,1.2,51.5,1.2,13.3,28.7
13,East South Central,21.5,0.6,17.8,0.8,Q,1.9,0.4,0.2,19.1,0.4,4.8,8.6
14,West South Central,29.3,1.3,52.6,2.5,Q,4.4,0.4,0.4,24.4,0.7,8.5,16.8


In [2]:
output_file("energy_consumption_bar_pandas_groupby_nested.html")


In [3]:
#Rename the column names
df2 = df2.rename(columns={'Unnamed: 1': 'Space_Heating', 'Unnamed: 2': 'Air_Handlers_for_Heating','Unnamed: 3': 'Air_Conditioning', 'Unnamed: 4': 'Air_Handlers_for_Cooling','Unnamed: 5': 'Evaporative_Coolers', 'Unnamed: 6': 'Ceiling_Fans','Unnamed: 7': 'Dehumidifiers', 'Unnamed: 8': 'Humidifiers','Unnamed: 9': 'Water_Heating', 'Unnamed: 10': 'Clothes_Washers','Unnamed: 11': 'Clothes_Drying', 'Unnamed: 12': 'Lighting', 'Release date: May 2018': 'Region'})
#To select rows whose column value equals a scalar, some_value, use ==:
array = ['New England', 'Middle Atlantic', 'East North Central', 'West North Central', 'South Atlantic', 'East South Central', 'West South Central', 'Mountain', 'Pacific']
df3= df2.loc[df2['Region'].isin(array)]

df3
df3.to_csv('out.csv')

In [4]:
df = pd.read_excel('End_Use_Consumption.xlsx')

df.head()


,mfr,mpg,cyl
0,Space heating,3.5,New England
1,Air handlers for heating,1.4,New England
2,Air conditioning,2.8,New England
3,Air handlers for cooling,0.1,New England
4,Evaporative coolers,0.0,New England


In [5]:
df.cyl = df.cyl.astype(str)


group = df.groupby(('cyl', 'mfr'))

source = ColumnDataSource(group)
index_cmap = factor_cmap('cyl_mfr', palette=Spectral10, factors=sorted(df.cyl.unique()), end=1)

p = figure(plot_width=1200, plot_height=600, title="Energy Usage by Region and End Use (Billions of kWh)",
           x_range=group, toolbar_location=None, tools="")

p.vbar(x='cyl_mfr', top='mpg_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, )

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "End Use Consumption Grouped by Region"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

p.add_tools(HoverTool(tooltips=[("Electricity Usage", "@mpg_mean"), ("Region, End Use", "@cyl_mfr")]))

show(p)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  after removing the cwd from sys.path.
